
Unsupervised Change Detection in Satellite Images
Using Principal Component Analysis
and k -Means Clustering

Atreya Majumdar


In [3]:
import cv2 as cv
import numpy as np
from matplotlib import pyplot as plt 
from sklearn.cluster import KMeans
import math

In [4]:
#path1 = "im1.jpg" #Read image 1 in Grayscale
#path2 = "im2.jpg" #Read image 2 in Grayscale

im1 = cv.imread("im1.jpg",0)
im2 = cv.imread("im2.jpg",0)

if im1.shape==im2.shape:  #Storing height and width of image if dimensions of two images are the same
    H, W = im1.shape 
else:
    H,W =im1.shape   #Resizing image 2 to dimensions of image 1 if they are different and then storing them
    im2=cv.resize(im2, (H,W) )
    im1=cv.resize(im1,(H,W))
    

In [5]:
def Normaldiffimg(im1,im2,H,W):  # Function to generate normalized difference image
    difference1 = cv.absdiff(im1, im2)
    norm_img = np.zeros((H,W))
    normalizedimg = cv.normalize(difference1,  norm_img, 0, 255, cv.NORM_MINMAX)
    return normalizedimg

In [6]:
diffimg= Normaldiffimg(im1,im2, H, W) #Generating normalized difference image from given images

In [7]:
#cv.imwrite("differenceatreya.png",diff_img)

In [8]:
#print(normalizedimg.shape)
#print(im1.shape)
#print(im2.shape)


In [9]:
def Blocks(diff_img,h, H, W): #Function to generate h x h non-overlapping blocks in image
    imgblock=[]
    for i in range(H):
        for j in range(W):
            imgblock.append(np.reshape(diff_img[i : i + h, j : j + h], (h**2, 1)))
    imgblock = np.array(imgblock)
    return imgblock

In [10]:
def co_variance_matrix(imgblock):
    pre_c=[]
    for i in imgblock[:5]:  #Computing the co-variance matrix, h^2 x h^2
        ele=np.array([i])
        x=np.dot(ele.T,ele) #Computing dot product of delta_p transpose and delta_p
        pre_c.append(x)
    new_pre=np.array(pre_c)
    return new_pre

In [11]:
def pca(matrix, imgblock):               #Applying Principal Component Analysis
    evals, evecs = np.linalg.eig(matrix) #Storing the eigen values and vectors of the co-variance matrix
    newc=evals.argsort()[::-1]           #Quicksorts the eigen vector matrixx then reverses it
    evals=evals[newc]
    evecs=evecs[newc]
    for i in range(len(evals)):
        if sum(evals[i:]) >= 0.9*sum(evals):  #Chose Rate= 0.9 here
            break
    new_evecs= evecs[:,i:]
    f_vec= np.dot(imgblock, new_evecs)  #Calculating feature vector
    return f_vec

In [12]:
def K_means_func(fv,H,W):   # Applying K-means clustering with 2 clusters (k=2) and detecting changes
    #Here, k=2
    fitter=KMeans(2).fit(fv).labels_
    change_map=np.reshape(fitter, (H,W))
    if sum(sum(change_map == 1)) > sum(sum(change_map == 0)):
        change_map[change_map == 1] = 0
        change_map[change_map == 0] = 1
    change_map = np.abs(change_map)*255
    change_map = change_map.astype(np.uint8)
    return change_map

In [13]:
def final_map(im1, im2, H, W, h):
    
    diffimg=Normaldiffimg(im1,im2,H,W)
    
    
    pads = int(np.ceil(h / 2)) 
    diffimg = np.pad(diffimg, ((pads, pads), (pads, pads)))  # Add padding to difference image
    
    imgblock = Blocks(diffimg,h, H, W)[:,:,0]
    M = int(W*H/h*h)  # np.floor() returns a float so directly converting to int with () is more efficient
    c_matrix = np.sum(co_variance_matrix(imgblock), axis = 0)/M 
    
    fv=pca(c_matrix,imgblock)
    change_map=K_means_func(fv,H,W)
    return change_map
    

In [14]:
h=4
cv.imwrite("Normalized_Difference_Img.png",diffimg)
computed_map=final_map(im1,im2,H,W,h)
cv.imwrite("Final_change_map.png",computed_map)



True